In [1]:
def get_metadata_features(i):
    """
    Obtain the dataset
    """
    # Extracted the text for nlp embeddings  
    df_X_train= pd.read_pickle('data-files\\X_train_' + str(i) + '.pkl')
    #print(df_X_train.head())
    df_X_test= pd.read_pickle('data-files\\X_test_' + str(i) + '.pkl')
    df_Y_train=pd.read_pickle('data-files\\Y_train_' + str(i) + '.pkl')
    df_Y_test=pd.read_pickle('data-files\\Y_test_' + str(i) + '.pkl')
    #print(df_X_train.shape) #(835, 5)
    #print(df_Y_train.shape) #(835, 1)
    #print(df_X_test.shape) #(209, 5)
    #print(df_Y_test.shape) #(209, 1)
    
    X_train_meta = df_X_train.drop(['Content', 'Previous_User_Utterance'], axis=1)
    X_test_meta = df_X_test.drop(['Content', 'Previous_User_Utterance'], axis=1)
    
    #print(X_train_meta['Previous_Speech_Act'].unique())
    #print(X_train_meta['Previous_Search_Act'].unique())
    
    return X_train_meta, X_test_meta, df_Y_train, df_Y_test

In [2]:
from keras.engine.topology import Layer

class AttentionLayer(Layer):
    
    def __init__(self,attention_dim=100,return_coefficients=False,**kwargs):
        # Initializer 
        self.supports_masking = True
        self.return_coefficients = return_coefficients
        self.init = initializers.get('glorot_uniform') # initializes values with uniform distribution
        self.attention_dim = attention_dim
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Builds all weights
        # W = Weight matrix, b = bias vector, u = context vector
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)),name='W')
        self.b = K.variable(self.init((self.attention_dim, )),name='b')
        self.u = K.variable(self.init((self.attention_dim, 1)),name='u')
        self.trainable_weights = [self.W, self.b, self.u]

        super(AttentionLayer, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, hit, mask=None):
        # Here, the actual calculation is done
        uit = K.bias_add(K.dot(hit, self.W),self.b)
        uit = K.tanh(uit)
        
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)
        ait = K.exp(ait)
        
        if mask is not None:
            ait *= K.cast(mask, K.floatx())

        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = hit * ait
        
        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), ait]
        else:
            return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[-1], 1)]
        else:
            return input_shape[0], input_shape[-1]

Using TensorFlow backend.


In [3]:
def create_bilstm_model_meta(reshaped_data):
    samples,timesteps,features = reshaped_data.shape
    input_data = Input(shape=(timesteps,features))
    # lstm needs (samples,timesteps,features) tensor as the input
    x = Bidirectional(LSTM(BATCH_SIZE, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(input_data) 
    #x = Bidirectional(LSTM(BATCH_SIZE, return_sequences=True, dropout=0.25))(input_data) 
    x, sent_coeffs = AttentionLayer(features,return_coefficients=True,name='sent_attention')(x)
    #x = GlobalMaxPool1D()(x)
    #x = Dense(100, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(12, activation="softmax")(x) #output layer
    model = Model(inputs=input_data, outputs=x)
    return model  

In [4]:
def execute_bilstm_metadata(i):
    df_train_meta, df_test_meta, df_Y_train, df_Y_test = get_metadata_features(i)
    train_meta = df_train_meta.values
    test_meta = df_test_meta.values
    print("Shape check post file read:")
    #print(train_meta)
    print(train_meta.shape, test_meta.shape)   
    
    timestamp = 1 #number of successive sequences combined
    
    r, c = df_train_meta.shape
    staggering = r - timestamp + 1 # final number of instances generated 
    X_train_reshaped = np.concatenate([train_meta[x:x+timestamp,:] for x in range(r-timestamp+1)])
    X_train_reshaped = X_train_reshaped.reshape(staggering, timestamp, c) # c is the number of features
    #print(X_train_reshaped)
    print("X_train_reshaped.shape:")
    print(X_train_reshaped.shape)
    
    r2, c2 = df_test_meta.shape
    staggering2 = r2 - timestamp + 1 # final number of instances generated 
    X_test_reshaped = np.concatenate([test_meta[x:x+timestamp,:] for x in range(r2-timestamp+1)])
    X_test_reshaped = X_test_reshaped.reshape(staggering2, timestamp, c2) # c is the number of features
    #print(X_test_reshaped)
    print("X_test_reshaped.shape:")
    print(X_test_reshaped.shape)
    
    df_Y_train = df_Y_train.iloc[timestamp-1:,]
    df_Y_test = df_Y_test.iloc[timestamp-1:,]
    
    print("Shape check for train and test sets post reshaping for lstm:")
    print(X_train_reshaped.shape, X_test_reshaped.shape)
    print(df_Y_train.shape, df_Y_test.shape)
    
    model = create_bilstm_model_meta(X_train_reshaped)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['binary_accuracy', 'categorical_accuracy'])
    model.summary()
    
    """
    encoding the output labels
    """
    encoder = LabelEncoder()
    encoder.fit(df_Y_train)
    training_op_labels_encoded = encoder.transform(df_Y_train)
    test_op_labels_encoded = encoder.transform(df_Y_test)
    print("Training and test output labels(before one-hot/after enc)")
    print(training_op_labels_encoded.shape, test_op_labels_encoded.shape)
    """
    converting the output labels to one-hot form
    """
    training_op_labels_onehot= np_utils.to_categorical(training_op_labels_encoded)
    test_op_labels_onehot = np_utils.to_categorical(test_op_labels_encoded)

    #print(training_op_labels_onehot)
    print("Training and test output labels(after one-hot)")
    print(training_op_labels_onehot.shape, len(training_op_labels_onehot))
    print(test_op_labels_onehot.shape, len(test_op_labels_onehot))

    print("X_train_reshaped", X_train_reshaped.shape)
    print("training_op_labels_onehot", training_op_labels_onehot.shape)
    model.fit(X_train_reshaped, training_op_labels_onehot, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1)

    """
    evaluate model
    """
    print(model.metrics_names)
    loss, binary_accuracy, categorical_accuracy = model.evaluate(X_test_reshaped, y = test_op_labels_onehot, batch_size=BATCH_SIZE, verbose=1)
    
    print(loss, binary_accuracy, categorical_accuracy)

    """
    predict the probabilty of output classes
    and pick the best one
    """
    prediction_vector = model.predict(X_test_reshaped, batch_size=BATCH_SIZE, verbose=1)
    predicted_classes = np.argmax(prediction_vector, axis=1)
    original_classes = np.argmax(test_op_labels_onehot, axis=1)
    
    """
    # verification of correctness:
    total_correct = sum(original_classes == predicted_classes)
    print("Total number of correct predictions:",total_correct)
    print("Accuracy:",total_correct/len(test_op_labels_onehot))
    acc = np.sum(conf_mat.diagonal()) / np.sum(conf_mat)
    print('Overall accuracy: {} %'.format(acc*100))
    """
    conf_mat = confusion_matrix(predicted_classes, original_classes)
    print(conf_mat)

    """
    # show the inputs and predicted outputs
    list_bilstm_prediction = []    
    for i in range(len(X_test_reshaped)):
        row = []
        row.extend(X_test_reshaped[i])
        row.extend(test_op_labels_onehot[i])
        list_bilstm_prediction.append(row)
    df_bilstm_prediction = pd.DataFrame(data=list_bilstm_prediction)
    df_bilstm_prediction.to_pickle("nlp_bilstm_predictions.pkl")
    """
       
    return predicted_classes, categorical_accuracy, binary_accuracy, conf_mat  

In [5]:
"""
import tensorflow as tf
tf_config = tf.compat.v1.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.9
tf_config.allow_soft_placement = True
"""

import pandas as pd
import numpy as np 
from numpy import array

from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.layers.merge import concatenate
from keras.models import Model, model_from_json
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import np_utils, to_categorical
from keras.engine.topology import Layer
from keras import backend as K

# checkpoint
from keras.callbacks import ModelCheckpoint

from tensorflow.keras.layers import Attention

BATCH_SIZE=32
EPOCHS=300

file = open('output\\meta-attn\\bilstm_meta_attn_op.txt','w') #overwrites previous
file.close()

if __name__== "__main__":
    df_prediction = pd.DataFrame()
    df_accuracy =  pd.DataFrame()
    file = open('output\\meta-attn\\bilstm_meta_attn_op.txt','a') #append mode 
    for i in range(1,31):
        outputname = 'meta_attn_'+ str(i)        
        predictions, acc_cat, acc_bin, conf_matrix = execute_bilstm_metadata(i)
        df_prediction[outputname] = predictions
        df_accuracy[i] = [acc_cat]
        file.write("\nIteration:" + str(i) + "\nCategorical Accuracy:" + str(acc_cat) + 
                   "\nBinary Accuracy:" + str(acc_bin) + "\nConfusion Matrix:\n" + str(conf_matrix) + "\n\n")
        df_prediction.to_csv('output\\meta-attn\\predictions_bilstm_attn_meta_' + str(i) + '.csv')    
        df_accuracy.to_csv('output\\meta-attn\\accuracy_bilstm_attn_meta_' + str(i) + '.csv') 
        
    df_prediction.to_csv('output\\meta-attn\\predictions_bilstm_attn_meta.csv')    
    df_accuracy.to_csv('output\\meta-attn\\accuracy_bilstm_attn_meta.csv')    
    file.close()

Shape check post file read:
(1467, 8) (367, 8)
X_train_reshaped.shape:
(1467, 1, 8)
X_test_reshaped.shape:
(367, 1, 8)
Shape check for train and test sets post reshaping for lstm:
(1467, 1, 8) (367, 1, 8)
(1467, 1) (367, 1)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
tracking <tf.Variable 'sent_attention/W:0' shape=(64, 8) dtype=float32> W
tracking <tf.Variable 'sent_attention/b:0' shape=(8,) dtype=float32> b
tracking <tf.Variable 'sent_attention/u:0' shape=(8, 1) dtype=float32> u
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 8)              0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1, 64)             10496     
_________________________________________________________________
sent_attention (AttentionLay [(None, 64), (None, 64, 1 528      

C:\Users\iSchool User\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)



Train on 1320 samples, validate on 147 samples
Epoch 1/300
1320/1320 [==============================] - 1s 397us/step - loss: 2.4099 - binary_accuracy: 0.9167 - categorical_accuracy: 0.1500 - val_loss: 2.1389 - val_binary_accuracy: 0.9167 - val_categorical_accuracy: 0.1905
Epoch 2/300
1320/1320 [==============================] - 0s 80us/step - loss: 2.1245 - binary_accuracy: 0.9167 - categorical_accuracy: 0.2553 - val_loss: 1.9133 - val_binary_accuracy: 0.9167 - val_categorical_accuracy: 0.3537
Epoch 3/300
1320/1320 [==============================] - 0s 67us/step - loss: 2.0222 - binary_accuracy: 0.9171 - categorical_accuracy: 0.2795 - val_loss: 1.8149 - val_binary_accuracy: 0.9167 - val_categorical_accuracy: 0.4354
Epoch 4/300
1320/1320 [==============================] - 0s 80us/step - loss: 1.9355 - binary_accuracy: 0.9179 - categorical_accuracy: 0.3114 - val_loss: 1.7694 - val_binary_accuracy: 0.9195 - val_categorical_accuracy: 0.4830
Epoch 5/300
1320/1320 [========================

Epoch 36/300
1320/1320 [==============================] - 0s 61us/step - loss: 1.4514 - binary_accuracy: 0.9259 - categorical_accuracy: 0.4917 - val_loss: 1.2569 - val_binary_accuracy: 0.9325 - val_categorical_accuracy: 0.5986
Epoch 37/300
1320/1320 [==============================] - 0s 60us/step - loss: 1.4129 - binary_accuracy: 0.9265 - categorical_accuracy: 0.5144 - val_loss: 1.2492 - val_binary_accuracy: 0.9320 - val_categorical_accuracy: 0.6122
Epoch 38/300
1320/1320 [==============================] - 0s 62us/step - loss: 1.4438 - binary_accuracy: 0.9266 - categorical_accuracy: 0.5053 - val_loss: 1.2545 - val_binary_accuracy: 0.9331 - val_categorical_accuracy: 0.5918
Epoch 39/300
1320/1320 [==============================] - 0s 63us/step - loss: 1.4466 - binary_accuracy: 0.9263 - categorical_accuracy: 0.4917 - val_loss: 1.2230 - val_binary_accuracy: 0.9314 - val_categorical_accuracy: 0.6054
Epoch 40/300
1320/1320 [==============================] - 0s 61us/step - loss: 1.4374 - bina

Epoch 108/300
1320/1320 [==============================] - 0s 64us/step - loss: 1.2978 - binary_accuracy: 0.9338 - categorical_accuracy: 0.5598 - val_loss: 1.1391 - val_binary_accuracy: 0.9422 - val_categorical_accuracy: 0.6327
Epoch 109/300
1320/1320 [==============================] - 0s 61us/step - loss: 1.2848 - binary_accuracy: 0.9331 - categorical_accuracy: 0.5470 - val_loss: 1.1680 - val_binary_accuracy: 0.9427 - val_categorical_accuracy: 0.6122
Epoch 110/300
1320/1320 [==============================] - 0s 62us/step - loss: 1.2872 - binary_accuracy: 0.9343 - categorical_accuracy: 0.5523 - val_loss: 1.1412 - val_binary_accuracy: 0.9427 - val_categorical_accuracy: 0.6599
Epoch 111/300
1320/1320 [==============================] - 0s 76us/step - loss: 1.2862 - binary_accuracy: 0.9352 - categorical_accuracy: 0.5644 - val_loss: 1.1605 - val_binary_accuracy: 0.9416 - val_categorical_accuracy: 0.6395
Epoch 112/300
1320/1320 [==============================] - 0s 73us/step - loss: 1.3046 -

...

Epoch 36/300
1320/1320 [==============================] - 0s 149us/step - loss: 1.4062 - binary_accuracy: 0.9268 - categorical_accuracy: 0.4902 - val_loss: 1.2808 - val_binary_accuracy: 0.9388 - val_categorical_accuracy: 0.5986
Epoch 37/300
1320/1320 [==============================] - 0s 146us/step - loss: 1.4277 - binary_accuracy: 0.9273 - categorical_accuracy: 0.5159 - val_loss: 1.2817 - val_binary_accuracy: 0.9365 - val_categorical_accuracy: 0.6054
Epoch 38/300
1320/1320 [==============================] - 0s 141us/step - loss: 1.4035 - binary_accuracy: 0.9269 - categorical_accuracy: 0.5038 - val_loss: 1.2689 - val_binary_accuracy: 0.9393 - val_categorical_accuracy: 0.5646
Epoch 39/300
1320/1320 [==============================] - 0s 153us/step - loss: 1.4201 - binary_accuracy: 0.9273 - categorical_accuracy: 0.5220 - val_loss: 1.2732 - val_binary_accuracy: 0.9399 - val_categorical_accuracy: 0.6190
Epoch 40/300
1320/1320 [==============================] - 0s 142us/step - loss: 1.4055 -

1320/1320 [==============================] - 0s 146us/step - loss: 1.2994 - binary_accuracy: 0.9331 - categorical_accuracy: 0.5477 - val_loss: 1.1977 - val_binary_accuracy: 0.9410 - val_categorical_accuracy: 0.6327
Epoch 105/300
1320/1320 [==============================] - 0s 142us/step - loss: 1.2825 - binary_accuracy: 0.9352 - categorical_accuracy: 0.5598 - val_loss: 1.1824 - val_binary_accuracy: 0.9427 - val_categorical_accuracy: 0.6599
Epoch 106/300
1320/1320 [==============================] - 0s 148us/step - loss: 1.2487 - binary_accuracy: 0.9350 - categorical_accuracy: 0.5636 - val_loss: 1.1926 - val_binary_accuracy: 0.9433 - val_categorical_accuracy: 0.6395
Epoch 107/300
1320/1320 [==============================] - 0s 142us/step - loss: 1.2592 - binary_accuracy: 0.9355 - categorical_accuracy: 0.5659 - val_loss: 1.1847 - val_binary_accuracy: 0.9433 - val_categorical_accuracy: 0.6599
Epoch 108/300
1320/1320 [==============================] - 0s 149us/step - loss: 1.2550 - binary_a

1320/1320 [==============================] - 0s 147us/step - loss: 1.2165 - binary_accuracy: 0.9367 - categorical_accuracy: 0.5818 - val_loss: 1.1677 - val_binary_accuracy: 0.9433 - val_categorical_accuracy: 0.6395
Epoch 174/300
1320/1320 [==============================] - 0s 136us/step - loss: 1.2037 - binary_accuracy: 0.9390 - categorical_accuracy: 0.5977 - val_loss: 1.1892 - val_binary_accuracy: 0.9399 - val_categorical_accuracy: 0.6259
Epoch 175/300
1320/1320 [==============================] - 0s 146us/step - loss: 1.2006 - binary_accuracy: 0.9371 - categorical_accuracy: 0.5803 - val_loss: 1.1834 - val_binary_accuracy: 0.9410 - val_categorical_accuracy: 0.6122
Epoch 176/300
1320/1320 [==============================] - 0s 148us/step - loss: 1.1989 - binary_accuracy: 0.9377 - categorical_accuracy: 0.5879 - val_loss: 1.1693 - val_binary_accuracy: 0.9439 - val_categorical_accuracy: 0.6599
Epoch 177/300
1320/1320 [==============================] - 0s 144us/step - loss: 1.2225 - binary_a

1320/1320 [==============================] - 0s 145us/step - loss: 1.1476 - binary_accuracy: 0.9405 - categorical_accuracy: 0.6144 - val_loss: 1.1591 - val_binary_accuracy: 0.9416 - val_categorical_accuracy: 0.6395
Epoch 244/300
1320/1320 [==============================] - 0s 144us/step - loss: 1.1691 - binary_accuracy: 0.9402 - categorical_accuracy: 0.6098 - val_loss: 1.1838 - val_binary_accuracy: 0.9439 - val_categorical_accuracy: 0.6327
Epoch 245/300
1320/1320 [==============================] - 0s 142us/step - loss: 1.1911 - binary_accuracy: 0.9388 - categorical_accuracy: 0.5992 - val_loss: 1.1529 - val_binary_accuracy: 0.9433 - val_categorical_accuracy: 0.6599
Epoch 246/300
1320/1320 [==============================] - 0s 147us/step - loss: 1.1493 - binary_accuracy: 0.9410 - categorical_accuracy: 0.6038 - val_loss: 1.1851 - val_binary_accuracy: 0.9427 - val_categorical_accuracy: 0.6190
Epoch 247/300
1320/1320 [==============================] - 0s 145us/step - loss: 1.1820 - binary_a

Epoch 279/300
1320/1320 [==============================] - 0s 147us/step - loss: 1.1418 - binary_accuracy: 0.9417 - categorical_accuracy: 0.6083 - val_loss: 1.1301 - val_binary_accuracy: 0.9444 - val_categorical_accuracy: 0.6531
Epoch 280/300
1320/1320 [==============================] - 0s 145us/step - loss: 1.1429 - binary_accuracy: 0.9403 - categorical_accuracy: 0.6182 - val_loss: 1.1683 - val_binary_accuracy: 0.9422 - val_categorical_accuracy: 0.6259
Epoch 281/300
1320/1320 [==============================] - 0s 144us/step - loss: 1.1747 - binary_accuracy: 0.9405 - categorical_accuracy: 0.6091 - val_loss: 1.1369 - val_binary_accuracy: 0.9433 - val_categorical_accuracy: 0.6463
Epoch 282/300
1320/1320 [==============================] - 0s 143us/step - loss: 1.1643 - binary_accuracy: 0.9400 - categorical_accuracy: 0.6023 - val_loss: 1.1627 - val_binary_accuracy: 0.9422 - val_categorical_accuracy: 0.6259
Epoch 283/300
1320/1320 [==============================] - 0s 144us/step - loss: 1.1